In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
#shuffle_files: The MNIST data is only stored in a single file,
#but for larger datasets with multiple files on disk,
#it's good practice to shuffle them when training.

#as_supervised: Returns tuple (img, label) instead of dict {'image': img, 'label': label}

In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#ds.map: TFDS provide the images as tf.uint8, while the model expect tf.float32, so normalize images
ds_train = ds_train.cache()
#ds.cache As the dataset fit in memory, cache before shuffling for better performance.
#Note: Random transformations should be applied after caching
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
#ds.shuffle: For true randomness, set the shuffle buffer to the full dataset size.
#Note: For bigger datasets which do not fit in memory, a standard value is 1000 if your system allows it.
ds_train = ds_train.batch(128)
#ds.batch: Batch after shuffling to get unique batches at each epoch.
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
#ds.prefetch: Good practice to end the pipeline by prefetching for performances.

In [4]:
#Testing pipeline is similar to the training pipeline, with small differences:
#in this case we dont call ds.suffle()
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
#Caching is done after batching (as batches can be the same between epoch)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
)

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3493 - sparse_categorical_accuracy: 0.9051 - val_loss: 0.2050 - val_sparse_categorical_accuracy: 0.9418
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1603 - sparse_categorical_accuracy: 0.9544 - val_loss: 0.1333 - val_sparse_categorical_accuracy: 0.9597
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1157 - sparse_categorical_accuracy: 0.9667 - val_loss: 0.1143 - val_sparse_categorical_accuracy: 0.9646
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0918 - sparse_categorical_accuracy: 0.9730 - val_loss: 0.0966 - val_sparse_categorical_accuracy: 0.9696
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0742 - sparse_categorical_accuracy: 0.9786 - val_loss: 0.0886 - val_sparse_categorical_accuracy: 0.9730
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0617 - sparse_categorica